In [1]:
import os
import pickle
import csv 
from datetime import datetime 
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from unidecode import unidecode

# --- PENGATURAN & KONFIGURASI ---

# Nama file kredensial OAuth 2.0.
CLIENT_SECRETS_FILE = "client_secret(unikom).json"

# Nama file untuk menyimpan log komentar yang dihapus.
LOG_FILE = 'deletion_log.csv'

# SCOPES menentukan tingkat izin yang Anda minta.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

# --- KATA KUNCI INDIKASI JUDI ONLINE ---
KATA_INDIKASI_JUDOL = [
    "dewa","777","77","88", "gacor", "slot gacor", "maxwin", "big win", "mega win", "super win","jepe","gajir","toto",
    "jackpot (jp)", "auto jp", "auto cuan", "winrate tinggi", "mudah menang", 
    "hoki parah", "hoki gila", "hoki sejati", "spin hoki", "turbo spin", 
    "buyspin hoki", "buyspin receh", "buyspin langsung jp", "scatter gacor", 
    "banjir scatter", "full scatter", "room gacor", "room sultan", "room sakti", 
    "room dewa", "trik gacor", "trik anti zonk", "settingan admin", 
    "settingan sakti", "settingan gacor", "modal receh, hasil sultan", 
    "modal 10k tembus 1jt", "receh jadi sultan", "cuan terus", "cuan gila", 
    "cuan nonstop", "cuan maksimal", "langsung cuan", "langsung profit", 
    "langsung maxwin", "langsung wd", "langsung kaya", "auto withdraw", 
    "auto win", "auto hoki", "auto sultan", "sultan mode", "sultan room", 
    "sultan spin", "badai cuan", "badai jp", "badai scatter", "ledakan jp", 
    "ledakan cuan", "petir bersinar", "petir cuan", "petir jp", "sensasional", 
    "gila roomnya", "gila cuan", "pecah scatter", "pecah jp", "tembus maxwin", 
    "tembus jackpot", "top win", "top player", "cair", "sultan player", 
    "jitu", "jitu hoki", "anti loss", "anti rungkad", "no rungkad", "bebas loss", 
    "pola gacor", "pola hari ini", "slot gacorr", "slot 77", "jp jam segini", 
    "slot hari ini", "jam hoki", "modal kecil profit", "dari 10k ke juta", 
    "transferan masuk", "isi saldo", "wd kilat", "wd cepat", "room jp", 
    "slot terbaik", "slot online", "slot anti rungkad", "petir menyambar", 
    "hujan cuan", "scatter melimpah", "bocoran slot", "pattern win", 
    "cuan slot", "admin settingan", "bot slot", "bot jp", "top cuan", 
    "sultan jp", "gila menang", "slot auto jp", "trik modal receh", 
    "auto profit", "cu4n", "sl0t", "s|ot", "slothoki", "cuan harian", 
    "viral slot", "trending slot", "modal 10k", "maxw1n", "makwin", "mw"
]

# --- FUNGSI-FUNGSI ---

def get_authenticated_service():
    """
    Menangani alur otentikasi OAuth 2.0. Membuka browser untuk izin saat pertama kali.
    Mengembalikan service object yang siap digunakan untuk semua aksi.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
            
    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, credentials=creds)

def log_deleted_comment(log_data):
    """
    FUNGSI BARU: Menyimpan detail komentar yang dihapus ke dalam file CSV.
    """
    file_exists = os.path.exists(LOG_FILE)
    with open(LOG_FILE, 'a', newline='', encoding='utf-8') as csvfile:
        # Nama kolom untuk file CSV telah disederhanakan.
        fieldnames = ['timestamp', 'video_id', 'comment_id', 'comment_text', 'reason']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(log_data)

def get_all_video_ids(youtube_service):
    """
    Mengambil semua ID video dari channel yang terotentikasi.
    """
    all_video_ids = []
    try:
        channels_response = youtube_service.channels().list(
            part='contentDetails',
            mine=True
        ).execute()
        
        if not channels_response.get('items'):
            print("Tidak dapat menemukan channel. Pastikan Anda login dengan akun YouTube yang benar.")
            return []

        uploads_playlist_id = channels_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        
        request = youtube_service.playlistItems().list(
            part='contentDetails',
            playlistId=uploads_playlist_id,
            maxResults=50
        )
        
        print("Mengambil daftar semua video dari channel Anda...")
        while request:
            response = request.execute()
            for item in response['items']:
                all_video_ids.append(item['contentDetails']['videoId'])
            request = youtube_service.playlistItems().list_next(request, response)

        print(f"Total ditemukan {len(all_video_ids)} video di channel Anda.")
        return all_video_ids

    except HttpError as e:
        print(f"Terjadi error saat mengambil daftar video: {e}")
        return []

def scan_and_delete_comments_on_video(youtube_service, video_id):
    """
    Memindai satu video, mencari komentar Judi Online (termasuk balasan), dan menghapusnya.
    """
    print(f"\n--- Memindai Video ID: {video_id} ---")
    comments_to_delete = []
    
    try:
        # Meminta 'replies' untuk mendapatkan balasan komentar juga
        request = youtube_service.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=100,
            textFormat="plainText"
        )
        
        while request:
            response = request.execute()
            for item in response['items']:
                # 1. Proses Komentar Utama (Top-Level Comment)
                top_level_comment = item['snippet']['topLevelComment']
                original_text = top_level_comment['snippet']['textDisplay']
                normalized_text = unidecode(original_text).lower()

                for keyword in KATA_INDIKASI_JUDOL:
                    if keyword in normalized_text:
                        print(f"  [TERDETEKSI] Judi Online di video {video_id} | Alasan: '{keyword}' | ID: {top_level_comment['id']}")
                        comments_to_delete.append({
                            'id': top_level_comment['id'], 
                            'reason': keyword, 
                            'text': original_text,
                            'video_id': video_id
                        })
                        break

                # 2. Proses Balasan Komentar (Replies), jika ada
                if 'replies' in item:
                    for reply in item['replies']['comments']:
                        original_text_reply = reply['snippet']['textDisplay']
                        normalized_text_reply = unidecode(original_text_reply).lower()

                        for keyword in KATA_INDIKASI_JUDOL:
                            if keyword in normalized_text_reply:
                                print(f"  [TERDETEKSI] Judi Online di video {video_id} | Alasan: '{keyword}' | ID: {reply['id']}")
                                comments_to_delete.append({
                                    'id': reply['id'], 
                                    'reason': keyword, 
                                    'text': original_text_reply,
                                    'video_id': video_id
                                })
                                break
            
            request = youtube_service.commentThreads().list_next(request, response)

    except HttpError as e:
        if "disabled comments" in str(e).lower():
            print(f"  [INFO] Komentar dinonaktifkan untuk video {video_id}. Melanjutkan ke video berikutnya.")
        else:
            print(f"  [ERROR] Gagal mengambil komentar untuk video {video_id}: {e}")
        return

    if not comments_to_delete:
        print(f"  [BERSIH] Tidak ada komentar Judi Online ditemukan di video {video_id}.")
        return

    print(f"  Menghapus {len(comments_to_delete)} komentar dari video {video_id}...")
    for comment_data in comments_to_delete:
        comment_id = comment_data['id']
        reason = comment_data['reason']
        try:
            youtube_service.comments().delete(id=comment_id).execute()
            print(f"    [BERHASIL] Komentar {comment_id} dihapus (alasan: '{reason}').")
            
            # Siapkan data untuk ditulis ke file log.
            log_entry = {
                'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'video_id': comment_data['video_id'],
                'comment_id': comment_id,
                'comment_text': comment_data['text'],
                'reason': reason
            }
            log_deleted_comment(log_entry)

        except HttpError as e:
            # Tangani kasus di mana komentar sudah dihapus atau tidak dapat diproses.
            if e.resp.status == 404:
                print(f"    [INFO] Komentar {comment_id} tidak ditemukan (kemungkinan sudah dihapus).")
            # Tangani error spesifik 'processingFailure' yang Anda laporkan.
            elif e.resp.status == 400 and 'processingFailure' in str(e.content):
                print(f"    [INFO] Gagal memproses {comment_id} (kemungkinan sudah dihapus oleh sistem YouTube).")
            else:
                # Untuk error lainnya, tampilkan pesan GAGAL standar.
                print(f"    [GAGAL] Tidak dapat menghapus komentar {comment_id}. Alasan: {e}")

# --- FUNGSI UTAMA ---
if __name__ == '__main__':
    try:
        print("Memulai proses moderasi channel...")
        youtube = get_authenticated_service()
        video_ids = get_all_video_ids(youtube)
        
        if video_ids:
            print("\n--- Memulai Pemindaian Komentar di Semua Video ---")
            for video_id in video_ids:
                scan_and_delete_comments_on_video(youtube, video_id)
            print("\n--- Semua video telah selesai dipindai. ---")
            print(f"Log penghapusan telah disimpan di file: {LOG_FILE}")
        else:
            print("Tidak ada video untuk dipindai.")
            
    except FileNotFoundError:
        print(f"\n[ERROR UTAMA] File '{CLIENT_SECRETS_FILE}' tidak ditemukan.")
        print("Pastikan Anda sudah men-download file kredensial dari Google Cloud Console,")
        print(f"mengganti namanya menjadi '{CLIENT_SECRETS_FILE}', dan menempatkannya di folder yang sama.")
    except Exception as e:
        print(f"\n[ERROR FATAL] Terjadi kesalahan yang tidak terduga: {e}")


Memulai proses moderasi channel...
Mengambil daftar semua video dari channel Anda...
Total ditemukan 2 video di channel Anda.

--- Memulai Pemindaian Komentar di Semua Video ---

--- Memindai Video ID: yLmqHgWlmO0 ---
  [TERDETEKSI] Judi Online di video yLmqHgWlmO0 | Alasan: '777' | ID: Ugzq3W6W8Oi7X782gVx4AaABAg
  [TERDETEKSI] Judi Online di video yLmqHgWlmO0 | Alasan: 'gacor' | ID: UgzqwrLKBIiBgbnwlJh4AaABAg
  Menghapus 2 komentar dari video yLmqHgWlmO0...
    [BERHASIL] Komentar Ugzq3W6W8Oi7X782gVx4AaABAg dihapus (alasan: '777').
    [BERHASIL] Komentar UgzqwrLKBIiBgbnwlJh4AaABAg dihapus (alasan: 'gacor').

--- Memindai Video ID: N3NmEBziagk ---
  [BERSIH] Tidak ada komentar Judi Online ditemukan di video N3NmEBziagk.

--- Semua video telah selesai dipindai. ---
Log penghapusan telah disimpan di file: deletion_log.csv
